In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import  IntegerType
from pyspark.sql.functions import udf

ss = SparkSession.builder.appName("Spark Data Frame Example").getOrCreate()
df = ss.read.options(inferSchema='True', header='True').csv('OfficeData.csv')

df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- bonus: integer (nullable = true)



NY: 10% of Sal + 5% of bonus
CA: 12% of Sal + 3% of bonus

In [4]:
def calculateIncrement(state,salary,bonus):
    sum = 0
    if(state == "NY"):
        sum =  (salary*.10) 
        sum += (bonus * .05)
    elif state =="CA":
        sum =  (salary*.12) 
        sum += (bonus * .03)
    return int(sum)

increment = udf(lambda x,y,z:calculateIncrement(x,y,z), IntegerType())
#df2 = df.withColumn("increment",lit(100))
df2 = df.withColumn("increment",increment(df.state,df.salary,df.bonus))
df2.show()

+-------------+----------+-----+------+---+-----+---------+
|employee_name|department|state|salary|age|bonus|increment|
+-------------+----------+-----+------+---+-----+---------+
|        James|     Sales|   NY| 90000| 34|10000|     9500|
|      Michael|     Sales|   NY| 86000| 56|20000|     9600|
|       Robert|     Sales|   CA| 81000| 30|23000|    10410|
|        Maria|   Finance|   CA| 90000| 24|23000|    11490|
|        Raman|   Finance|   CA| 99000| 40|24000|    12600|
|        Scott|   Finance|   NY| 83000| 36|19000|     9250|
|          Jen|   Finance|   NY| 79000| 53|15000|     8650|
|         Jeff| Marketing|   CA| 80000| 25|18000|    10140|
|        Kumar| Marketing|   NY| 91000| 50|21000|    10150|
+-------------+----------+-----+------+---+-----+---------+



In [5]:
df2.cache()

DataFrame[employee_name: string, department: string, state: string, salary: int, age: int, bonus: int, increment: int]

In [7]:
df2.show()

+-------------+----------+-----+------+---+-----+---------+
|employee_name|department|state|salary|age|bonus|increment|
+-------------+----------+-----+------+---+-----+---------+
|        James|     Sales|   NY| 90000| 34|10000|     9500|
|      Michael|     Sales|   NY| 86000| 56|20000|     9600|
|       Robert|     Sales|   CA| 81000| 30|23000|    10410|
|        Maria|   Finance|   CA| 90000| 24|23000|    11490|
|        Raman|   Finance|   CA| 99000| 40|24000|    12600|
|        Scott|   Finance|   NY| 83000| 36|19000|     9250|
|          Jen|   Finance|   NY| 79000| 53|15000|     8650|
|         Jeff| Marketing|   CA| 80000| 25|18000|    10140|
|        Kumar| Marketing|   NY| 91000| 50|21000|    10150|
+-------------+----------+-----+------+---+-----+---------+

